In [58]:
import json
import glob
import pandas as pd
import matplotlib.pyplot as plt
import scipy

pd.set_option('display.max_rows', 30)

In [59]:
def read(path):
    with open(path, "r") as f:

        content = json.load(f)
        data = content['_checkpoint']['records']
    return data

In [60]:
def create_df(data):

    df = pd.json_normalize(data)

    # remove prefices from column name
    df.columns = df.columns.str.removeprefix('meta.')
    df.columns = df.columns.str.removeprefix('scores.')
    # df = df.rename(columns={"index": "route_index"})

    interesting_columns = ['route_id', 'status', 'duration_game', 'duration_system',
                           'route_length', 'score_composed', 'score_penalty', 'score_route']
    df = df[interesting_columns]

    # Strip prefix
    df['route_id'] = df['route_id'].str.removeprefix("RouteScenario_")

    # Make a numerical column
    df['route_id'] = pd.to_numeric(df['route_id'])

    # normalize data to be in range [0, max]
    df['route_id'] -= df['route_id'].min()
    return df

In [61]:
path = "results/experiment2/2_paralel_quality_epic_fps_10_repeat_3.json"
df_epic = create_df(read(path))

path = "results/experiment2/2_paralel_quality_low_fps_10_repeat_3.json"
df_low = create_df(read(path))

## Paralelized

In [68]:
DURATION = df_low['duration_system'].sum()

sum_system_hours = DURATION/3600
print(f"{sum_system_hours:f} real hours")

sum_game_hours = (df_low['duration_game'].sum() +
                  df_low['duration_game'].sum()) / 3600
print(f"{sum_game_hours:f} simulated hours")
ratio_parelel = sum_game_hours/sum_system_hours
print(f"Ratio {ratio_parelel:f}")

15.774654 real hours
11.621556 simulated hours
Ratio 0.736723


In [63]:
path = "results/experiment2/quality_low_fps_10_repeat_3.json"
df_baseline_low = create_df(read(path))

In [64]:
df_baseline_low

,route_id,status,duration_game,duration_system,route_length,score_composed,score_penalty,score_route
0,0,Completed,829.700012,1682.781150,1071.028134,28.619907,0.286199,100.000000
1,0,Completed,707.000011,1417.550364,1071.028134,60.000000,0.600000,100.000000
2,0,Completed,771.700011,1422.287662,1071.028134,100.000000,1.000000,100.000000
3,1,Failed - Agent timed out,714.100011,831.435408,862.430089,78.987766,1.000000,78.987766
4,1,Failed - Agent timed out,714.100011,819.948063,862.430089,34.326845,0.360000,95.352348
5,1,Failed - Agent timed out,714.100011,829.152907,862.430089,47.456818,0.600000,79.094697
6,2,Failed - Agent timed out,846.100013,1979.549502,1018.154809,26.543327,0.600000,44.238878
7,2,Completed,628.900009,1205.393605,1018.154809,60.000000,0.600000,100.000000
8,2,Completed,621.100009,1152.758113,1018.154809,100.000000,1.000000,100.000000
9,3,Completed,1099.300016,1232.027795,1651.124043,0.768774,0.007688,100.000000


In [65]:
df_joined = df_low.join(df_baseline_low, how="inner", rsuffix=".baseline")

In [66]:
cols = ['duration_system', 'duration_system.baseline']
df_joined.groupby("route_id")[cols].mean()

,duration_system,duration_system.baseline
route_id,,
0,3184.568722,1507.539725
1,1385.310050,826.845459
2,2340.250837,1445.900407
3,2738.429769,1402.471894
4,1751.852829,1090.518710
5,846.486362,486.836992
6,1863.468399,1136.262566
7,3194.883567,1905.590535
8,2436.500699,1599.476236


## Not paralelized

In [69]:
DURATION = df_baseline_low['duration_system'].sum()

sum_system_hours = DURATION/3600
print(f"{sum_system_hours:f} real hours")

sum_game_hours = df_baseline_low['duration_game'].sum() / 3600
print(f"{sum_game_hours:f} simulated hours")
ratio_baseline = sum_game_hours/sum_system_hours
print(f"Ratio {ratio_baseline:2f}")

11.441427 real hours
7.110389 simulated hours
Ratio 0.621460
